In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()) + '/source')

import numpy as np
import SolveLinSys
from utilities import compute_derivatives

Equation:

$$
0 = \max_{e} \min_{h_2} - \delta \phi(r, z_2) + \delta \eta \log e - \tau z_2 e - \frac{\partial \phi}{\partial r}(r, z_2) e + \xi_m\frac{(h_2)^2}{2} + \left[\frac{\partial \phi}{\partial z_2}(r, z_2)\right]\left[-\rho (z_2 -\mu_2) + \sqrt{z_2} \sigma_2 h_2\right] + \left[\frac{\partial^2 \phi}{\partial(z_2)^2}(r, z_2)\right]\left(\frac{z_2|\sigma_2|^2}{2}\right)
$$

FOC of $h_2$ gives : 

$$
h_2^* = -\frac{\left[\frac{\partial \phi}{\partial z_2}(r, z_2)\right]\sqrt{z_2}\sigma_2}{\xi_m}
$$

FOC of $e$ gives :

$$
e^* = \frac{\delta \eta}{\tau z_2 + \frac{\partial \phi}{\partial r}(r, z_2)}
$$

In [ ]:
def false_transient_one_iteration(stateSpace, A, B_r, B_z, C_rr, C_zz, D, v0, ε=.3, tol=-10):
    A = A.reshape(-1, 1, order='F')
    B = np.hstack([B_r.reshape(-1, 1, order='F'), B_z.reshape(-1, 1, order='F')])
    C = np.hstack([C_rr.reshape(-1, 1, order='F'), C_zz.reshape(-1, 1, order='F')])
    D = D.reshape(-1, 1, order='F')
    v0 = v0.reshape(-1, 1, order='F')
    out = SolveLinSys.solveFT(stateSpace, A, B, C, D, v0, ε, tol)
    return out


def false_transient(z_grid, r_grid, model_paras=(), v0=None, ϵ=.3, tol=1e-8, max_iter=10_000):
    τ, η, ξ_m, δ, η, μ_2, ρ, σ_2 = model_paras

    Δ_z = z_grid[1] - z_grid[0]
    Δ_r = r_grid[1] - r_grid[0]
    (z_mat, r_mat) = np.meshgrid(z_grid, r_grid, indexing = 'ij')
    stateSpace = np.hstack([z_mat.reshape(-1, 1, order='F'), r_mat.reshape(-1, 1, order='F')])

    if v0 is None:
        v0 = δ*η*r_mat

    count = 1
    error = 1.

    while error > tol and count < max_iter:
        print('Iteration:{:d}'.format(count))
        v_old = v0.copy()
        v0_dz = compute_derivatives(v0, 0, 1, Δ_z)
        v0_dzz = compute_derivatives(v0, 0, 2, Δ_z)
        v0_dr = compute_derivatives(v0, 1, 1, Δ_r)

        e = δ * η / (τ*z_mat + v0_dr)
        h = -v0_dz*np.sqrt(z_mat)*σ_2/ξ_m

        A = -δ*np.ones_like(z_mat)
        B_z = -ρ*(z_mat - μ_2) + np.sqrt(z_mat)*σ_2*h
        B_r = -e
        C_zz = z_mat*σ_2**2/2
        C_rr = np.zeros_like(z_mat)
        D = δ*η*np.log(e) - τ*z_mat*e + ξ_m*h**2/2

        res = false_transient_one_iteration(stateSpace, A, B_z, B_r, C_zz, C_rr, D, v0, ϵ) # CG tol = 1e-10
        v0 = res[2].reshape(v0.shape, order = "F")

        rhs_error = A*v0 + B_z*v0_dz + B_r*v0_dr + C_zz*v0_dzz + D
        rhs_error = np.max(abs(rhs_error))
        lhs_error = np.max(abs((v0 - v_old)/ϵ))
        error = np.max([lhs_error, rhs_error])

        print("Iteration %s: LHS Error: %s; RHS Error %s" % (count, lhs_error, rhs_error))
        count += 1
    return v0, e

In [ ]:
# Model parameters
median = 1.75/1000
γ = .018
τ = median * γ
η = .032
ξ_m = .00256
δ = .01
η = .032

μ_2 = 1.
ρ = .5
σ_2 = np.sqrt((.21)**2*2*ρ/μ_2) # Match moments, using 100 year's std

model_paras = (τ, η, ξ_m, δ, η, μ_2, ρ, σ_2)

# Grid setting
n_z = 100
z_min = 1e-5
z_max = 2
z_grid = np.linspace(z_min, z_max, n_z)

n_r = 200
r_min = 0
r_max = 3000
r_grid = np.linspace(r_min, r_max, n_r)

In [ ]:
ϕ, e = false_transient(z_grid, r_grid, model_paras=model_paras, v0=None, ϵ=.5, tol=1e-9, max_iter=10_000)